# SPARQL Workshop - Section 01: Select Queries & Filter Patterns 

In this section, you will start with some basic SPARQL queries that are used to find nodes (vertices) and relations (properties/edges) with given attributes.  Those attributes can be labels (or classes in RDF), unique IDs, or properties (also called "predicates" in RDF).  Feel free to change any of the queries in this (and subsequent) notebooks to see how changing the query may result in different outcomes.

Happy Traversing!

## Graph Model

As we've mentioned prior, we will be using a modified version of the IMDb (Internet Movie Database) dataset for this workshp.  We have converted this data into the following graph data model representing certain aspects of the data as nodes (vertices), relations (edges), and properties.  While not part of this workshop, it is important to understand graph data modeling.  Please refer to our Graph Reference Architectures GitHub repo for more details on how to perform graph data modeling:  https://github.com/aws-samples/aws-dbs-refarch-graph

Here's the application graph data model:

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-rdf-updated.png"/>



## Select a node and its Properties

When selecting graph elements, bear the following in mind:

 - Neptune allows you to supply URIs to identify nodes in the graph. Use these if possible to lookup resources (nodes, properties, edge URIs)
 - If you can't use an URI to lookup an resource, use the _minimum number_ of label and property predicates necessary to uniquely identify the element or the set of elements you want to find.
 - When projecting results, select only the variables necessary to satisfy your query, rather than projecting out all the bindings for the variables satisfying the pattens.

### 01.01 - A Quick Review of the RDF Data Model

When using RDF and SPARQL, each aspect of the data model is stored as a triple - subject, predicate, object.  Each subject and predicate is referenced as a URI. Objects can be referenced as URIs or literals (e.g. string, integer). The URI schema that is used can be arbitrary, or it can be resolvable to resources in the web.  Before we get into how to write SPARQL queries in more detail, it is important to know how the pictorial version of the data model above relates to the URI schema that was derived to store the data as triples.

The following query will show all of the unique types used for this dataset:

In [ ]:
%%sparql

# Select the different entity types in our dataset

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?type

WHERE 
{ 
    ?s rdf:type ?type 
}

You'll notice how the "types" relate directly the the vertex labels associated with the pictoral view of our data model.

Next, let's take a look a look at the predicates/properties available in our data model:

In [ ]:
%%sparql

# Select the different predicates in our dataset

SELECT DISTINCT ?predicate

WHERE 
{ 
    ?subject ?predicate ?object 
}

You'll notice that each of the property names (or predicates) listed above relate to all of the property names in the pictoral version of our data model.  If you look at each of the URIs in the above two queries, you should notice that they relate to the following unique URIs:

- http://www.w3.org/1999/02/22-rdf-syntax-ns#type
- http://www.w3.org/2000/01/rdf-schema#label
- http://aws.amazon.com/neptune/csv2rdf/class/
- http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/
- http://aws.amazon.com/neptune/csv2rdf/objectProperty/
- http://aws.amazon.com/neptune/csv2rdf/resource/

The first two URIs that contain `w3.org` are pre-defined RDF types that are compabitle across many RDF data stores.  The next three URIs are defined as per the namespace we have defined for our sample dataset.  `class` dictates the type of the RDF resource (vertex type).  `datatypeProperty` denotes a property that will have a literal as its value. `objectProperty` denotes a property that will contain another URI.  In most circumstances, you could think of `objectProperty` as defining an edge between two vertices in the graph.  There is also `resource`, which defines the URIs of each vertex.

### 01.02 - Find a resource by URI

To start, you will perform the most simple of SPARQL operations to find a resource by its unique URI.  Each RDF resource has a unique URI. URIs for resources can be assigned using a common scheme of namespace prefix + a unique resource Id. Examples of this unique id can be email addresses signifying a person resource.  In the IMDb dataset, IDs are already provided for both Artists and their works (e.g. movies, TV shows, etc.).  For example, if you go to the [IMDb page for Kevin Bacon](https://www.imdb.com/name/nm0000102/) you will notice the last part of the URL is `nm0000102` is the unique identifer for Kevin.

In the two queries in section `01.01`, you may have noticed we used the keyword `PREFIX`.  `PREFIX` is used to create short-hand notation for URIs used in a query.  Instead of having to include full URIs, we can create shortcuts for these.  You'll notice in the following example that we use `resource` as a shortcut to `http://aws.amazon.com/neptune/csv2rdf/resource/`.

In the following example, we use Kevin Bacon's identifer to find his vertex and return on predicates and values related to his vertex:

In [ ]:
%%sparql

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>
PREFIX prop: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

SELECT * 

WHERE 
{
    resource:nm0000102 ?p ?o
} 

Similar to our query on Kevin Bacon, we can perform the same query using one of Kevin's more famous movies, "Footloose".  Just for the sake of example, we know that the unique identifier of "Footloose" is `tt0087277`.  Let's try finding all properties related to "Footloose".:

In [ ]:
%%sparql

PREFIX resource: <http://aws.amazon.com/neptune/csv2rdf/resource/>

SELECT * 
WHERE 
{
    resource:tt0087277 ?p ?o
} 

### 01.03 - Filtering by Certain Properties

Sometimes you won't know the exact ID for a specific resource you are looking for.  It is more likely that you would know the value of a given property associated for the resource of interest.  In those cases, we can query on the specific value (or object) and look for the related subject(s).

In the below example, we want to find all vertices representing productions with the title of "Footloose".

In [ ]:
%%sparql

PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

SELECT ?subject
WHERE 
{
    ?subject dataProp:title "Footloose" .
} 

This particular query returns all URIs (or vertex IDs) for movies with title "Footloose".  Notice that it does not return any other values related to the movie.  Why?

For one, "Footloose" is the title to a lot of different productions (not just the two feature motion pictures, but a number of TV shows and the like).  But by querying via a single triple pattern with "Footloose" in the object field and a defined predicate (dataProp:title), we will only see subjects related to the production's title.  We're going to need to add some additional triple patterns to our query to make this more complete.  Let's take a look at the following query.

In [ ]:
%%sparql

PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

SELECT ?movie ?predicate ?value 

WHERE 
{
    ?movie rdf:type class:Movie .
    ?movie dataProp:title "Footloose" .
    ?movie ?predicate ?value
} 

You should see output representing the two renditions of the movie "Footloose".  One from 1984 and the more recent from 2011.

By adding the triples `?movie rdf:type class:Movie` and `?movie ?predicate ?values` it both narrows the focus of the query to only those resources that are movie productions (removes all TV shows, etc.) and includes all subject-predicate-objects that are related to the two renditions of "Footloose".

### 01.04 - Using Conditionals

Beyond doing simple `SELECT-WHERE` queries, there are a few other ways to find resources and properties based on certain search criteria.  For example, say we wanted to find all instances of "Footloose", movies or otherwise, that were filmed before 2005.  In this case, we can use the `FILTER` command within SPARQL to do this.

To find all versions of "Footloose" prior to 2005 we can use the following:

In [ ]:
%%sparql

PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

SELECT ?production ?year
WHERE 
{
    ?production dataProp:title "Footloose" .
    ?production dataProp:year ?year .
    FILTER (?year < 2005)
} 

We can also combine multiple `FILTER` conditions by using the `&&` for AND or `||` for OR operators.

For example, if we wanted to find all versions of Footloose filmed from 2000 to 2012:

In [ ]:
%%sparql

PREFIX class: <http://aws.amazon.com/neptune/csv2rdf/class/>
PREFIX dataProp: <http://aws.amazon.com/neptune/csv2rdf/datatypeProperty/>

SELECT ?production ?year
WHERE {
    ?production dataProp:title "Footloose" .
    ?production dataProp:year ?year .
    FILTER (?year > 2000 && ?year < 2012)
} 

### Review

In the previous notebook we covered the basics of using SPARQL to look at individual entities and their subsequent predicates (properties).  We also looked at how to begin filtering on predicate values.  We'll begin to build on these simple queries in the following notebooks and use these patterns as starting and ending conditions for traversing our graph.

### End of Section 01 - Move onto the next notebook.